# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [ ]:
pip install gmaps

In [12]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import gmaps

# Import API key
from api_keys import (geoapify_key,gkey)

In [13]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,atuona,-9.8000,-139.0333,25.17,81,31,6.31,PF,1667534564
1,1,addis abeba,9.0250,38.7469,10.49,76,44,1.14,ET,1667534401
2,2,bredasdorp,-34.5322,20.0403,17.69,94,97,5.37,ZA,1667534624
3,3,belmonte,-15.8631,-38.8828,20.36,90,100,7.48,BR,1667534624
4,4,bafoulabe,13.8065,-10.8321,22.26,21,0,1.15,ML,1667534625


In [14]:
locations = city_data_df[["Lat", "Lng"]]
locations

,Lat,Lng
0,-9.8000,-139.0333
1,9.0250,38.7469
2,-34.5322,20.0403
3,-15.8631,-38.8828
4,13.8065,-10.8321
...,...,...
557,28.5004,-13.8627
558,2.7717,-77.6654
559,-33.3667,116.1500
560,30.7554,20.2263


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [15]:
%%capture --no-display

# Configure gmaps with API key
gmaps.configure(api_key=gkey)

# Configure the map plot

#city_data_df = city_data_df.dropna()
locations = city_data_df[["Lat", "Lng"]]
#locations
humidity = city_data_df["Humidity"]
#humidity
# Display the map
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [16]:
# Narrow down cities that fit criteria and drop any results with null values
farenheit_temps = (city_data_df["Max Temp"] * 1.8) + 32
#type(farenheit_temps)

city_merged_df=city_data_df.merge(farenheit_temps.rename('Farenheit Max Temp'),
             left_index=True, right_index=True)

#city_merged_df

ideal_weather_df = city_merged_df.loc[(city_merged_df["Farenheit Max Temp"] >= 73) & (city_merged_df["Farenheit Max Temp"] <= 82)]
#ideal_weather_df

# Drop any rows with null values
ideal_weather_df = ideal_weather_df.dropna()

# Display sample data
ideal_weather_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Farenheit Max Temp
0,0,atuona,-9.8000,-139.0333,25.17,81,31,6.31,PF,1667534564,77.306
11,11,butaritari,3.0707,172.7902,27.01,69,75,5.43,KI,1667534627,80.618
13,13,avarua,-21.2078,-159.7750,24.03,69,40,5.14,CK,1667534628,75.254
19,19,salinopolis,-0.6136,-47.3561,25.71,85,65,3.27,BR,1667534629,78.278
21,21,vaini,-21.2000,-175.2000,27.09,78,20,3.60,TO,1667534630,80.762


### Step 3: Create a new DataFrame called `hotel_df`.

In [17]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_df[['City', 'Country', 'Lat','Lng','Humidity']].copy()

#hotel_df
# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = None

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
0,atuona,PF,-9.8000,-139.0333,81,None
11,butaritari,KI,3.0707,172.7902,69,None
13,avarua,CK,-21.2078,-159.7750,69,None
19,salinopolis,BR,-0.6136,-47.3561,85,None
21,vaini,TO,-21.2000,-175.2000,78,None


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [18]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "apiKey": geoapify_key,
    "categories": "accommodation.hotel"
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    #print (lat)
    lng = row["Lng"]
    #print(lng)
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"
    
    #print (params)
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"
    
    #response = requests.get(base_url, params=params)

    # Make and API request using the params dictionaty
    #name_address = # YOUR CODE HERE
    
    # Convert the API response to JSON format
    name_address = requests.get(base_url, params=params).json()
    
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
atuona - nearest hotel: Pearl Resort
butaritari - nearest hotel: No hotel found
avarua - nearest hotel: Paradise Inn
salinopolis - nearest hotel: Hotel Salinópolis
vaini - nearest hotel: Keleti International Resort
hilo - nearest hotel: Dolphin Bay Hotel
mahebourg - nearest hotel: Grand Bel Air
travesia - nearest hotel: No hotel found
kudahuvadhoo - nearest hotel: No hotel found
cockburn town - nearest hotel: The Salt Raker Inn
kumbhraj - nearest hotel: No hotel found
saint-louis - nearest hotel: No hotel found
inhambane - nearest hotel: Hotel de Inhambane
kapaa - nearest hotel: Pono Kai Resort
sao filipe - nearest hotel: Bela Vista
pecos - nearest hotel: Holiday Inn Express & Suites Pecos
san patricio - nearest hotel: No hotel found
muyuka - nearest hotel: No hotel found
kloulklubed - nearest hotel: The Adventures Inn
tura - nearest hotel: No hotel found
bambous virieux - nearest hotel: Otentic Eco Tent Experience
hambantota - nearest hotel: Oasis Aurweda Resort 

,City,Country,Lat,Lng,Humidity,Hotel Name
0,atuona,PF,-9.8000,-139.0333,81,Pearl Resort
11,butaritari,KI,3.0707,172.7902,69,No hotel found
13,avarua,CK,-21.2078,-159.7750,69,Paradise Inn
19,salinopolis,BR,-0.6136,-47.3561,85,Hotel Salinópolis
21,vaini,TO,-21.2000,-175.2000,78,Keleti International Resort
...,...,...,...,...,...,...
519,salalah,OM,17.0151,54.0924,73,Muscat International Hotel
522,navolato,MX,24.7672,-107.6944,84,Hotel Maioris Navolato
532,iralaya,HN,15.0000,-83.2333,85,No hotel found
537,qui nhon,VN,13.7667,109.2333,73,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [19]:
%%capture --no-display

# Configure the map plot

hotel_df_list= hotel_df["Hotel Name"].tolist()

marker_locations = hotel_df[['Lat','Lng']]

# Create a marker_layer using the poverty list to fill the info box
fig = gmaps.figure()
markers = gmaps.marker_layer(marker_locations,
    info_box_content=[f"Hotel: {hotel}" for hotel in hotel_df_list])
fig.add_layer(markers)
##fig

# Display the map
fig

Figure(layout=FigureLayout(height='420px'))